Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше


In [8]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 72.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=45d163666d2c35b4feb85ec5de8f67dd788d582050d47aceda6cf433de7b9300
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [35]:
import numpy as np
import pandas as pd

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

In [2]:
data = pd.read_excel('отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [3]:
max_words = 20000
max_len = 150
num_classes = 5

epochs = 20
batch_size = 512
print_batch_n = 100

In [18]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

morpher = MorphAnalyzer()

def words_only(text):
    text=str(text)
    return text.lower()   

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [13]:
data['normalized_content'] = data['Content'].apply(normalize_text)
data.head()

,Rating,Content,Date,normalized_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать 1 работа антивирус далёкий нику...
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро


In [16]:
train_corpus = " ".join(data['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [19]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:7]

['приложение', 'очень', 'удобно', 'работать', 'удобный', 'вс', 'спасибо']

In [20]:
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [21]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [24]:
data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in data['normalized_content']], 
    dtype=np.int32)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)

In [28]:
X_train.shape

(14461, 150)

In [30]:
le = LabelEncoder()

y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)

le.classes_

array([1, 2, 3, 4, 5])

### 2.1 предобученные эмбединги

In [31]:
!wget http://vectors.nlpl.eu/repository/20/220.zip
!unzip /content/220.zip

--2023-01-31 19:06:53--  http://vectors.nlpl.eu/repository/20/220.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638171816 (609M) [application/zip]
Saving to: ‘220.zip’

220.zip             100%[===================>] 608.61M  26.5MB/s    in 23s     

2023-01-31 19:07:17 (26.0 MB/s) - ‘220.zip’ saved [638171816/638171816]

Archive:  /content/220.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [32]:
embeddings_index = {}
with open('/content/model.txt') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 249334 word vectors.


In [33]:
embedding_dim = 200

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [36]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    weights=[embedding_matrix], 
                    output_dim=embedding_dim, 
                    input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [37]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [38]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.1)

Epoch 1/10
26/26 [==============================] - 23s 855ms/step - loss: 1.5971 - accuracy: 0.6834 - val_loss: 1.5832 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 22s 839ms/step - loss: 1.5714 - accuracy: 0.7051 - val_loss: 1.5576 - val_accuracy: 0.7091
Epoch 3/10
26/26 [==============================] - 23s 887ms/step - loss: 1.5464 - accuracy: 0.7051 - val_loss: 1.5327 - val_accuracy: 0.7091
Epoch 4/10
26/26 [==============================] - 22s 838ms/step - loss: 1.5220 - accuracy: 0.7051 - val_loss: 1.5085 - val_accuracy: 0.7091
Epoch 5/10
26/26 [==============================] - 22s 859ms/step - loss: 1.4983 - accuracy: 0.7051 - val_loss: 1.4850 - val_accuracy: 0.7091
Epoch 6/10
26/26 [==============================] - 23s 850ms/step - loss: 1.4753 - accuracy: 0.7051 - val_loss: 1.4621 - val_accuracy: 0.7091
Epoch 7/10
26/26 [==============================] - 22s 844ms/step - loss: 1.4530 - accuracy: 0.7051 - val_loss: 1.4400 - val_accuracy: 0.7091

In [39]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 182ms/step - loss: 1.3787 - accuracy: 0.7073
Test score: 1.378679871559143
Test accuracy: 0.7073249220848083


### 2.2 эмбединги по умолчанию

In [40]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())
model.add(Flatten())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [41]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.1)

Epoch 1/10
26/26 [==============================] - 17s 613ms/step - loss: 1.1852 - accuracy: 0.6946 - val_loss: 0.9810 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 13s 517ms/step - loss: 0.8704 - accuracy: 0.7051 - val_loss: 0.7634 - val_accuracy: 0.7139
Epoch 3/10
26/26 [==============================] - 13s 514ms/step - loss: 0.6920 - accuracy: 0.7583 - val_loss: 0.6802 - val_accuracy: 0.7588
Epoch 4/10
26/26 [==============================] - 13s 515ms/step - loss: 0.6120 - accuracy: 0.7842 - val_loss: 0.6572 - val_accuracy: 0.7664
Epoch 5/10
26/26 [==============================] - 15s 577ms/step - loss: 0.5623 - accuracy: 0.8003 - val_loss: 0.6526 - val_accuracy: 0.7713
Epoch 6/10
26/26 [==============================] - 14s 521ms/step - loss: 0.5237 - accuracy: 0.8193 - val_loss: 0.6542 - val_accuracy: 0.7789
Epoch 7/10
26/26 [==============================] - 13s 517ms/step - loss: 0.4882 - accuracy: 0.8343 - val_loss: 0.6543 - val_accuracy: 0.7789

In [43]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 122ms/step - loss: 0.6980 - accuracy: 0.7665
Test score: 0.6980157494544983
Test accuracy: 0.766537606716156


Точность на предобученных эмбедингах ниже. 